# Machine Learning Experiment \#1

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

import datetime
import math
import bisect
import time

In [ ]:
datafile = r'''..\Data\IVE_bidask1min.txt'''

colnames = ['Date', 'Time', 'BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh', 'AskLow', 'AskClose']
fullpricedata = pd.read_csv(datafile, names=colnames)

fullpricedata['DateTime'] = (fullpricedata['Date']+fullpricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
del fullpricedata['Date']
del fullpricedata['Time']

fullpricedata = fullpricedata[[(dt >= datetime.datetime(dt.year, dt.month, dt. day, 9, 30, 0)) and (dt <= datetime.datetime(dt.year, dt.month, dt. day, 16, 0, 0)) for dt in fullpricedata['DateTime']]].copy()

In [ ]:
pricedata = fullpricedata[fullpricedata['DateTime'] > datetime.datetime(2019,1,2,0,0,0)].copy()
#pricedata = pricedata[pricedata['DateTime'] < datetime.datetime(2019,1,3,0,0,0)].copy()
pricedata = pricedata.reset_index()
del pricedata['index']

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras import optimizers
from keras import regularizers
import random
import numpy as np
import pandas as pd
from operator import add


class DQNAgent(object):

    def __init__(self):
        self.inputs = 20
        self.outputs = 1
        self.model = self.network()

    def network(self, weights=None):
        model = Sequential()
        
        num_layers = 3
        neurons_per_layer = 20
        dropout = 0.0
        reg = 0.00
        act = 'tanh'
        output_act = 'linear'
        learning = 0.0025
        opt = optimizers.Adam(learning)
        
        if num_layers == 0:
            if dropout > 0:
                    model.add(Dropout(dropout))
            model.add(Dense(units=self.outputs, activation=output_act, input_dim=self.inputs,
                        kernel_regularizer=regularizers.l1(reg),
                        activity_regularizer=regularizers.l1(reg)))
        else:
            model.add(Dense(units=neurons_per_layer, activation=act, input_dim=self.inputs,
                            kernel_regularizer=regularizers.l1(reg),
                            activity_regularizer=regularizers.l1(reg)))
            
            for ii in range(num_layers - 1):
                if dropout > 0:
                    model.add(Dropout(dropout))
                model.add(Dense(units=neurons_per_layer, activation=act,
                                kernel_regularizer=regularizers.l1(reg),
                                activity_regularizer=regularizers.l1(reg)))
                
            if dropout > 0:
                    model.add(Dropout(dropout))
            model.add(Dense(units=self.outputs, activation=output_act,
                            kernel_regularizer=regularizers.l1(reg),
                            activity_regularizer=regularizers.l1(reg))) 
    
        model.compile(loss='mse', optimizer=opt)

        return model    
        
        
    def train(self, inputs, outputs):
        return self.model.fit(inputs, outputs, epochs=3000, verbose=2, batch_size=100)

In [ ]:
agent = DQNAgent()

inputs = []
rewards = []

start_date = pricedata['DateTime'][0].date()
end_date = pricedata['DateTime'][len(pricedata)-1].date()
days = []

curr_day = start_date
day_index = 0
while curr_day < end_date:
    days.append(curr_day)
    curr_day = curr_day + datetime.timedelta(days=1)

for day in days:
    daypricedata = pricedata[pricedata['DateTime'] > pd.Timestamp(day)].copy()
    daypricedata = daypricedata[daypricedata['DateTime'] < pd.Timestamp(day) + datetime.timedelta(days=1)].copy()
    daypricedata = daypricedata.reset_index()
    N = agent.inputs+1
    n = 10
    M = len(daypricedata)-N-n

    for index in range(0,M):
        inputs.append(np.diff(daypricedata['BidClose'][index:index+N]))
        rewards.append(daypricedata['BidClose'][index+N+n] - daypricedata['BidClose'][index+N])

    
inputs = np.array(inputs)
history = agent.train(inputs,rewards)
losses = history.history['loss']

predictions = agent.model.predict_on_batch(inputs)

In [ ]:
x_plot = [ii for ii in range(len(predictions))]
pred_plot = [w[0] for w in predictions]

pred_mean = np.mean(pred_plot)
rewards_mean = np.mean(rewards)
print(pred_mean, rewards_mean)

In [ ]:
class Chart:
    def __init__(self):
        data = [dict(
                type = 'scatter',
                x = x_plot,
                y = rewards,
                mode='lines'),
                dict(
                type = 'scatter',
                x = x_plot,
                y = pred_plot,
                mode='lines'),
               ]
        
        layout = dict(
                title = 'Plot')
        
        self.chart = go.FigureWidget( data=data, layout=layout )
        
        self.display = self.chart
        
chart = Chart()
chart.display

In [ ]:
class LossesChart:
    def __init__(self):
        data = [dict(
                type = 'scatter',
                x = [ii for ii in range(len(losses))],
                y = losses,
                mode='lines')
               ]
        
        layout = dict(
                title = 'Plot')
        
        self.chart = go.FigureWidget( data=data, layout=layout )
        
        self.display = self.chart
        
losses_chart = LossesChart()
losses_chart.display

# To-Do List

* Extract volume per minute data from tick data
* LossesChart/etc. applied to separate test data
* Try feeding a vector of indicators instead of raw price data
* Implement actual buy/sell strategy based on NN output, evaluate performance